In [10]:
import pandas as pd
import numpy as np
import json

In [11]:
traindf = pd.read_csv('../data/raw/data_set_phase1/train_queries.csv')
testdf = pd.read_csv('../data/raw/data_set_phase1/test_queries.csv')
planstr = pd.read_csv('../data/raw/data_set_phase1/train_plans.csv')
planste = pd.read_csv('../data/raw/data_set_phase1/test_plans.csv')

In [12]:
def generate_coordinates(df):
    df[["o_long", "o_lat"]] = df.o.str.split(",", 1, expand=True).astype(float)
    df.drop("o", axis=1, inplace=True)

    df[["d_long", "d_lat"]] = df.d.str.split(",", 1, expand=True).astype(float)
    df.drop("d", axis=1, inplace=True)
    
    return df

In [13]:
traindf = generate_coordinates(traindf)
testdf = generate_coordinates(testdf)

In [14]:
testdf.shape

(94358, 7)

In [15]:
alltr = pd.merge(traindf,planstr,how='outer')
alltr_test = pd.merge(testdf,planste,how='outer')

In [16]:
alltr.sample(5)

,sid,pid,req_time,o_long,o_lat,d_long,d_lat,plan_time,plans
280417,3394853,NaN,2018-11-24 10:41:59,116.42,39.99,116.37,39.86,2018-11-24 10:41:59,"[{""distance"": 21588, ""price"": 500, ""eta"": 4096..."
159572,2265932,131146.0,2018-10-13 11:15:14,116.25,40.12,116.30,39.92,2018-10-13 11:15:14,"[{""distance"": 39192, ""price"": 800, ""eta"": 6279..."
392539,85474,135987.0,2018-10-31 19:22:35,116.49,39.76,116.33,39.91,2018-10-31 19:22:35,"[{""distance"": 38443, ""price"": 1000, ""eta"": 705..."
285294,2627571,NaN,2018-10-12 23:19:56,116.34,40.04,116.39,39.90,2018-10-12 23:19:56,"[{""distance"": 19484, ""price"": """", ""eta"": 2313,..."
236796,2249091,NaN,2018-10-13 09:31:18,116.30,39.83,116.33,40.07,2018-10-13 09:31:18,"[{""distance"": 39408, ""price"": 2000, ""eta"": 462..."


In [17]:
alltr.shape

(500000, 9)

In [19]:
num_modes = 12
modes = []
for i in range(num_modes):
    modes.append('dist_' + str(i))
    modes.append('price_' + str(i))
    modes.append('eta_' + str(i))
    

In [20]:
num_modes = 12
for mode in modes:
    alltr[mode] = 0

In [21]:
num_modes = 12
for mode in modes:
    alltr_test[mode] = 0

In [123]:
alltr.sample(5)

,sid,pid,req_time,o_long,o_lat,d_long,d_lat,plan_time,plans,dist_0,...,eta_8,dist_9,price_9,eta_9,dist_10,price_10,eta_10,dist_11,price_11,eta_11
432583,3081068,151695.0,2018-11-16 13:58:09,116.27,40.00,116.40,40.06,2018-11-16 13:58:09,"[{""distance"": 25779, ""price"": 600, ""eta"": 4011...",0,...,0,0,0,0,0,0,0,0,0,0
250697,2230206,141933.0,2018-10-13 14:15:00,116.27,39.99,116.27,40.00,2018-10-13 14:15:00,"[{""distance"": 2782, ""price"": 2000, ""eta"": 785,...",0,...,0,0,0,0,0,0,0,0,0,0
315888,3094060,132831.0,2018-11-16 17:03:10,116.35,40.07,116.33,39.99,2018-11-16 17:03:10,"[{""distance"": 15335, ""price"": 500, ""eta"": 2036...",0,...,0,0,0,0,0,0,0,0,0,0
325264,2138527,206158.0,2018-10-19 11:04:01,116.15,39.72,116.36,39.92,2018-10-19 11:04:01,"[{""distance"": 36681, ""price"": 900, ""eta"": 5756...",0,...,0,0,0,0,0,0,0,0,0,0
415345,2371248,111552.0,2018-10-10 21:55:51,116.64,39.91,116.49,39.98,2018-10-10 21:55:51,"[{""distance"": 24887, ""price"": 900, ""eta"": 5144...",0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
alltr.iloc[0].plans

'[{"distance": 5219, "price": 300, "eta": 1367, "transport_mode": 9}, {"distance": 4537, "price": "", "eta": 1035, "transport_mode": 3}, {"distance": 4537, "price": 2100, "eta": 1095, "transport_mode": 4}, {"distance": 4118, "price": "", "eta": 1243, "transport_mode": 6}, {"distance": 5219, "price": 300, "eta": 1679, "transport_mode": 2}, {"distance": 4046, "price": "", "eta": 3672, "transport_mode": 5}]'

In [22]:
df_last = alltr.copy()

In [25]:
df_test_last = alltr_test.copy()

In [23]:
def preprocess_plans_last(df):
    for i, r in df.iterrows():
        if (i+1) % 5000 == 0:
            print("Processing row {}".format(str(i + 1)), end="\r")
        if isinstance(r.plans, float):
            # nan
            continue
        visited = []
        for pl in json.loads(r.plans):
            if pl['transport_mode'] in visited:
                continue
            visited.append(pl['transport_mode'])
            df.at[i,'dist_' + str(pl['transport_mode'])] = pl['distance']
            if pl['price']:
                df.at[i,'price_' + str(pl['transport_mode'])] = pl['price']
            else:
                df.at[i,'price_' + str(pl['transport_mode'])] = 999
            df.at[i,'eta_' + str(pl['transport_mode'])] = pl['eta']
    return df

In [195]:
def preprocess_plans(df):
    for i, r in df.iterrows():
        if (i+1) % 5000 == 0:
            print("Processing row {}".format(str(i + 1)), end="\r")
        if isinstance(r.plans, float):
            # nan
            continue
        for pl in json.loads(r.plans):
            df.at[i,'dist_' + str(pl['transport_mode'])] = pl['distance']
            if pl['price']:
                df.at[i,'price_' + str(pl['transport_mode'])] = pl['price']
            else:
                df.at[i,'price_' + str(pl['transport_mode'])] = 999
            df.at[i,'eta_' + str(pl['transport_mode'])] = pl['eta']
    return df

In [24]:
alldf_last = preprocess_plans_last(df_last)

In [26]:
alldf_test_last = preprocess_plans_last(df_test_last)

In [185]:
alldf = preprocess_plans(alltr)

In [200]:
alldf_test = preprocess_plans(alltr_test)

In [187]:
alldf[modes].sample(20)

,dist_0,price_0,eta_0,dist_1,price_1,eta_1,dist_2,price_2,eta_2,dist_3,...,eta_8,dist_9,price_9,eta_9,dist_10,price_10,eta_10,dist_11,price_11,eta_11
455807,0,0,0,6404,400,3533,7709,300,3098,7765,...,0,0,0,0,0,0,0,8137,500,3078
297842,0,0,0,14107,400,4246,0,0,0,17378,...,0,16438,500,3220,0,0,0,0,0,0
290048,0,0,0,1096,200,1280,0,0,0,2684,...,0,0,0,0,0,0,0,0,0,0
442248,0,0,0,6499,200,2828,0,0,0,5164,...,0,0,0,0,0,0,0,0,0,0
1813,0,0,0,839,200,1129,0,0,0,2744,...,0,0,0,0,0,0,0,0,0,0
270409,0,0,0,17163,400,5206,0,0,0,17057,...,0,17701,500,3140,0,0,0,0,0,0
290743,0,0,0,0,0,0,8853,400,2808,7992,...,0,0,0,0,0,0,0,0,0,0
244879,0,0,0,0,0,0,0,0,0,783,...,0,0,0,0,0,0,0,0,0,0
351248,0,0,0,0,0,0,11306,400,2556,9233,...,0,11311,400,2097,0,0,0,0,0,0
365118,0,0,0,60544,1700,11249,59477,800,7468,60918,...,0,59912,800,6933,61744,3500,6547,0,0,0


In [190]:
alldf.shape

(500000, 45)

In [191]:
alldf.to_pickle('train_preprocess.pickle')

In [201]:
alldf_test.to_pickle('test_preprocess.pickle')

In [27]:
alldf_last.to_pickle('train_preprocess_last.pickle')

In [28]:
alldf_test_last.to_pickle('test_preprocess_last.pickle')